In [ ]:
#|default_exp forecast

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# Forecast

> Full pipeline encapsulation

In [ ]:
#|export
from typing import Callable, Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn.base import RegressorMixin, clone

from mlforecast.core import TimeSeries
from mlforecast.utils import backtest_splits

In [ ]:
#|hide
import warnings

from nbdev import show_doc
from sklearn import set_config

In [ ]:
#|hide
set_config(display='text')
warnings.simplefilter('ignore', UserWarning)

In [ ]:
#|export
class Forecast:
    """Full pipeline encapsulation."""
    
    def __init__(
        self,
        models: Union[RegressorMixin, List[RegressorMixin]],  # model or list of models that follow the scikit-learn API
        freq: Optional[str] = None,  # pandas offset alias, e.g. D, W, M
        lags: List[int] = [],  # list of lags to use as features
        lag_transforms: Dict[int, List[Tuple]] = {},  # list of transformations to apply to each lag
        date_features: List[Union[str, Callable]] = [],  # list of names of pandas date attributes or functions to use as features, e.g. dayofweek
        differences: Optional[List[int]] = None,  # differences to apply to the series before fitting
        num_threads: int = 1,  # number of threads to use when computing lag features
    ):
        if not isinstance(models, list):
            models = [clone(models)]
        self.models = [clone(m) for m in models]
        self.ts = TimeSeries(freq, lags, lag_transforms, date_features, differences, num_threads)
        
    def __repr__(self):
        return (
            f'Forecast(models=[{", ".join(m.__class__.__name__ for m in self.models)}], '
            f'freq={self.freq}, '
            f'lag_features={list(self.ts.transforms.keys())}, '
            f'date_features={self.ts.date_features}, '
            f'num_threads={self.ts.num_threads})'
        )
    
    @property
    def freq(self):
        return self.ts.freq

    def preprocess(
        self,
        data: pd.DataFrame,  # dataframe with the series' data
        id_col: str = 'index',  # column that identifies each serie, can also be the index.
        time_col: str = 'ds',  # column with the timestamps
        target_col: str = 'y',  # column with the series values
        static_features: Optional[List[str]] = None,  # column names of the features that don't change in time
        dropna: bool = True,  # drop rows with missing values created by lags
        keep_last_n: Optional[int] = None,  # keep only this many observations of each serie for computing the updates    
    ) -> pd.DataFrame:
        return self.ts.fit_transform(data, id_col, time_col, target_col, static_features, dropna, keep_last_n)

    def fit(
        self,
        data: pd.DataFrame,  # dataframe with the series' data
        id_col: str = 'index',  # column that identifies each serie. If 'index', the index is taken as the identifier of each serie
        time_col: str = 'ds',  # column with the timestamps
        target_col: str = 'y',  # column with the series values
        static_features: Optional[List[str]] = None,  # column names of the features that don't change in time
        dropna: bool = True,  # drop rows with missing values created by lags
        keep_last_n: Optional[int] = None,  # keep only this many observations of each serie for computing the updates
    ) -> 'Forecast':
        """Preprocesses `data` and fits `models` using it."""
        series_df = self.preprocess(data, id_col, time_col, target_col, static_features, dropna, keep_last_n)
        X, y = series_df.drop(columns=[time_col, target_col]), series_df[target_col].values
        del series_df
        self.models_ = []
        for i, model in enumerate(self.models):
            self.models_.append(clone(model).fit(X, y))
        return self

    def predict(
        self,
        horizon: int,  # number of periods to predict in the future
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn
    ) -> pd.DataFrame:
        """Compute the predictions for the next `horizon` steps.
        
        `predict_fn(model, new_x, dynamic_dfs, features_order, **predict_fn_kwargs)` is called in every timestep, where:
        `model` is the trained model.
        `new_x` is a dataframe with the same format as the input plus the computed features.
        `dynamic_dfs` is a list containing the dynamic dataframes.
        `features_order` is the list of column names that were used in the training step.
        """
        return self.ts.predict(
            self.models_, horizon, dynamic_dfs, predict_fn, **predict_fn_kwargs
        )

    def cross_validation(
        self,
        data: pd.DataFrame,  # time series
        n_windows: int,  # number of windows to evaluate
        window_size: int,  # test size in each window
        id_col: str = 'index',  # column that identifies each serie, can also be the index.
        time_col: str = 'ds',  # column with the timestamps
        target_col: str = 'y',  # column with the series values
        static_features: Optional[List[str]] = None,  # column names of the features that don't change in time
        dropna: bool = True,  # drop rows with missing values created by lags
        keep_last_n: Optional[int] = None,  # keep only this many observations of each serie for computing the updates        
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,  # future values for dynamic features
        predict_fn: Optional[Callable] = None,  # custom function to compute predictions
        **predict_fn_kwargs,  # additional arguments passed to predict_fn
    ):
        """Creates `n_windows` splits of `window_size` from `data`, trains the model
        on the training set, predicts the window and merges the actuals and the predictions
        in a dataframe.

        Returns a dataframe containing the datestamps, actual values, train ends and predictions."""
        results = []
        self.cv_models_ = []
        if id_col != 'index':
            data = data.set_index(id_col)
        
        if np.issubdtype(data[time_col].dtype.type, np.integer):
            freq = 1
        else:
            freq = self.freq

        for train_end, train, valid in backtest_splits(data, n_windows, window_size, freq, time_col, target_col):
            self.fit(train, 'index', time_col, target_col, static_features, dropna, keep_last_n)
            self.cv_models_.append(self.models_)
            y_pred = self.predict(
                window_size, dynamic_dfs, predict_fn, **predict_fn_kwargs
            )
            y_pred = y_pred.set_index(time_col, append=True)
            result = valid.set_index(time_col, append=True)[[target_col]].copy()
            result = result.join(y_pred).reset_index(time_col)
            result['cutoff'] = train_end            
            results.append(result)

        out = pd.concat(results)
        out = out[[time_col, 'cutoff', target_col, *y_pred.columns]]
        if id_col != 'index':
            out = out.reset_index()
        return out

The `Forecast` class is a high level abstraction that encapsulates all the steps in the pipeline (preprocessing, fitting the model and computing the predictions). It tries to mimic the scikit-learn API.

## Example
This shows an example with simulated data, for a real world example you can check the [M5 example](https://www.kaggle.com/lemuz90/m5-mlforecast).

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

from mlforecast.utils import generate_daily_series, generate_prices_for_series

In order to forecast some time series you need a dataframe with at least three columns:

* `id_col`: contains the identifier for each time serie (can also be the index).
* `time_col`: contains the timestamps.
* `target_col`: contains the series values.

### Data setup

In [ ]:
series = generate_daily_series(100, equal_ends=True, n_static_features=2, static_as_categorical=False)
series

,ds,y,static_0,static_1
unique_id,,,,
id_00,2000-10-05,3.981198,79,45
id_00,2000-10-06,10.327401,79,45
id_00,2000-10-07,17.657474,79,45
id_00,2000-10-08,25.898790,79,45
id_00,2000-10-09,34.494040,79,45
...,...,...,...,...
id_99,2001-05-10,45.340051,69,35
id_99,2001-05-11,3.022948,69,35
id_99,2001-05-12,10.131371,69,35


Whatever extra columns you have, like `static_0` and `static_1` here are considered to be static and are replicated when constructing the features for the next datestamp. You can disable this by passing `static_features` to `Forecast.preprocess` or `Forecast.fit` , which will only keep the columns you define there as static. Keep in mind that they will still be used for training, so you'll have to provide them to `Forecast.predict` through the `dynamic_dfs` argument.

### Creating the Forecast object

The models can be any scikit-learn compatible regressors.

In [ ]:
models = [
    lgb.LGBMRegressor(n_jobs=1, random_state=0),
    xgb.XGBRegressor(n_jobs=1, random_state=0)
]

fcst = Forecast(
    models=models,
    freq='D',
    lags=[7],
    lag_transforms={
        1: [expanding_mean],
        7: [(rolling_mean, 14)]
    },
    date_features=['dayofweek', 'month'],
    num_threads=2,
)
fcst

Forecast(models=[LGBMRegressor, XGBRegressor], freq=<Day>, lag_features=['lag-7', 'expanding_mean_lag-1', 'rolling_mean_lag-7_window_size-14'], date_features=['dayofweek', 'month'], num_threads=2)

Here where we say that:

* Our series have daily frequency.
* We want to use lag 7 as a feature
* We want the lag transformations to be:
   * expanding mean of the lag 1
   * rolling mean of the lag 7 over a window of size 14
* We want to use dayofweek and month as date features.
* We want to perform the preprocessing and the forecasting steps using 2 threads.

From this point we have two options:

1. Preprocess the data and fit our models using all of it.
2. Preprocess the data and get it back as a dataframe to do some custom splitting or adding additional features, then training the models.

#### 1. Using all the data

In [ ]:
show_doc(Forecast.fit)

---

### Forecast.fit

>      Forecast.fit (data:pandas.core.frame.DataFrame, id_col:str='index',
>                    time_col:str='ds', target_col:str='y',
>                    static_features:Optional[List[str]]=None, dropna:bool=True,
>                    keep_last_n:Optional[int]=None)

Preprocesses `data` and fits `models` using it.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| data | DataFrame |  | dataframe with the series' data |
| id_col | str | index | column that identifies each serie. If 'index', the index is taken as the identifier of each serie |
| time_col | str | ds | column with the timestamps |
| target_col | str | y | column with the series values |
| static_features | typing.Optional[typing.List[str]] | None | column names of the features that don't change in time |
| dropna | bool | True | drop rows with missing values created by lags |
| keep_last_n | typing.Optional[int] | None | keep only this many observations of each serie for computing the updates |
| **Returns** | **Forecast** |  |  |

Calling `fit` on our data performs the preprocessing and uses all the data to train our model.

In [ ]:
fcst.fit(series)

Forecast(models=[LGBMRegressor, XGBRegressor], freq=<Day>, lag_features=['lag-7', 'expanding_mean_lag-1', 'rolling_mean_lag-7_window_size-14'], date_features=['dayofweek', 'month'], num_threads=2)

### Forecasting

In [ ]:
show_doc(Forecast.predict)

---

### Forecast.predict

>      Forecast.predict (horizon:int,
>                        dynamic_dfs:Optional[List[pandas.core.frame.DataFrame]]
>                        =None, predict_fn:Optional[Callable]=None,
>                        **predict_fn_kwargs)

Compute the predictions for the next `horizon` steps.

`predict_fn(model, new_x, dynamic_dfs, features_order, **predict_fn_kwargs)` is called in every timestep, where:
`model` is the trained model.
`new_x` is a dataframe with the same format as the input plus the computed features.
`dynamic_dfs` is a list containing the dynamic dataframes.
`features_order` is the list of column names that were used in the training step.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| horizon | int |  | number of periods to predict in the future |
| dynamic_dfs | typing.Optional[typing.List[pandas.core.frame.DataFrame]] | None | future values for dynamic features |
| predict_fn | typing.Optional[typing.Callable] | None | custom function to compute predictions |
| predict_fn_kwargs |  |  |  |
| **Returns** | **DataFrame** |  | **additional arguments passed to predict_fn** |

Once we have trained our models, we can compute the forecasts for the next 7 timesteps.

In [ ]:
fcst.predict(7)

,ds,LGBMRegressor,XGBRegressor
unique_id,,,
id_00,2001-05-15,42.144818,42.934929
id_00,2001-05-16,49.965606,51.280045
id_00,2001-05-17,2.133553,1.858219
id_00,2001-05-18,10.288427,9.754323
id_00,2001-05-19,18.616018,17.950321
...,...,...,...
id_99,2001-05-17,44.348196,43.529274
id_99,2001-05-18,1.947602,1.856401
id_99,2001-05-19,9.033580,8.921874


This uses each prediction as the next value of the target and updates all features accordingly. The static features were propagated and the date features were computed using each new timestamp.

In [ ]:
#|hide
preds = fcst.predict(7)
preds2 = fcst.predict(7)

pd.testing.assert_frame_equal(preds, preds2)

In [ ]:
#|hide
non_std_series = series.copy()
non_std_series['ds'] = non_std_series.groupby('unique_id').cumcount()
non_std_series = non_std_series.reset_index().rename(columns={'unique_id': 'some_id', 'ds': 'time', 'y': 'value'})
flow_params = dict(
    models=models,
    lags=[7],
    lag_transforms={
        1: [expanding_mean],
        7: [(rolling_mean, 14)]
    },
    num_threads=2,
)
fcst = Forecast(**flow_params)
non_std_preds = fcst.fit(non_std_series, id_col='some_id', time_col='time', target_col='value').predict(7)
non_std_preds.index.name = 'unique_id'
fcst = Forecast(freq='D', **flow_params)
preds = fcst.fit(series).predict(7)
pd.testing.assert_frame_equal(preds.drop(columns='ds'), non_std_preds.drop(columns='time'))

#### 2. Preprocess and train

If we only want to perform the preprocessing step we call `preprocess` with our data.

In [ ]:
show_doc(Forecast.preprocess)

---

### Forecast.preprocess

>      Forecast.preprocess (data:pandas.core.frame.DataFrame,
>                           id_col:str='index', time_col:str='ds',
>                           target_col:str='y',
>                           static_features:Optional[List[str]]=None,
>                           dropna:bool=True, keep_last_n:Optional[int]=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| data | DataFrame |  | dataframe with the series' data |
| id_col | str | index | column that identifies each serie, can also be the index. |
| time_col | str | ds | column with the timestamps |
| target_col | str | y | column with the series values |
| static_features | typing.Optional[typing.List[str]] | None | column names of the features that don't change in time |
| dropna | bool | True | drop rows with missing values created by lags |
| keep_last_n | typing.Optional[int] | None | keep only this many observations of each serie for computing the updates |
| **Returns** | **DataFrame** |  |  |

In [ ]:
features_df = fcst.preprocess(series)
features_df.head()

,ds,y,static_0,static_1,lag-7,expanding_mean_lag-1,rolling_mean_lag-7_window_size-14
unique_id,,,,,,,
id_00,2000-10-25,49.766844,79,45,50.694639,25.001367,26.320060
id_00,2000-10-26,3.918347,79,45,3.887780,26.180675,26.313387
id_00,2000-10-27,9.437778,79,45,11.512774,25.168751,26.398056
id_00,2000-10-28,17.923574,79,45,18.038498,24.484796,26.425272
id_00,2000-10-29,26.754645,79,45,24.222859,24.211411,26.305563


This is useful if we want to inspect the data the models will be trained on, adding additional features or performing some custom train-valid split. Here we perform an 80-20 split.

In [ ]:
np.random.seed(0)

train_mask = np.random.rand(features_df.shape[0]) < 0.8
train, valid = features_df[train_mask], features_df[~train_mask]
X_train, y_train = train.drop(columns=['ds', 'y']), train.y
X_valid, y_valid = valid.drop(columns=['ds', 'y']), valid.y

If we do this we must "manually" train our models and store them in the `models_` attribute.

In [ ]:
fitted = models[0].fit(
    X_train, y_train, 
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_names=['train', 'valid'],
    eval_metric='rmse',
    verbose=0
)
fcst.models_ = [fitted]

We can see the RMSE by iteration for each set.

In [ ]:
pd.DataFrame({
    k: np.round(fcst.models_[0].evals_result_[k]['rmse'], 2)
    for k in ('train', 'valid')
})

,train,valid
0,13.41,13.40
1,12.08,12.07
2,10.88,10.88
3,9.81,9.81
4,8.84,8.84
...,...,...
95,0.93,0.99
96,0.93,0.99
97,0.93,0.99
98,0.93,0.99


### Dynamic features

By default the predict method repeats the static features and updates the transformations and the date features. If you have dynamic features like prices or a calendar with holidays you can pass them as a list to the `dynamic_dfs` argument of `Forecast.predict`, which will call `pd.DataFrame.merge` on each of them in order.

Here's an example:

Suppose that we have a `product_id` column and we have a catalog for prices based on that `product_id` and the date.

In [ ]:
dynamic_series = series.rename(columns={'static_1': 'product_id'})
prices_catalog = generate_prices_for_series(dynamic_series)
prices_catalog

,ds,product_id,price
0,2000-06-09,1,0.548814
1,2000-06-10,1,0.715189
2,2000-06-11,1,0.602763
3,2000-06-12,1,0.544883
4,2000-06-13,1,0.423655
...,...,...,...
20180,2001-05-17,99,0.223520
20181,2001-05-18,99,0.446104
20182,2001-05-19,99,0.044783
20183,2001-05-20,99,0.483216


And you have already merged these prices into your series dataframe.

In [ ]:
series_with_prices = dynamic_series.reset_index().merge(prices_catalog, how='left')
series_with_prices.set_index('unique_id', inplace=True)
series_with_prices

,ds,y,static_0,product_id,price
unique_id,,,,,
id_00,2000-10-05,3.981198,79,45,0.570826
id_00,2000-10-06,10.327401,79,45,0.260562
id_00,2000-10-07,17.657474,79,45,0.274048
id_00,2000-10-08,25.898790,79,45,0.433878
id_00,2000-10-09,34.494040,79,45,0.653738
...,...,...,...,...,...
id_99,2001-05-10,45.340051,69,35,0.792152
id_99,2001-05-11,3.022948,69,35,0.782687
id_99,2001-05-12,10.131371,69,35,0.019463


This dataframe will be passed to `Forecast.fit` (or `Forecast.preprocess`), however since the price is dynamic we have to tell that method that only `static_0` and `product_id` are static and we'll have to update `price` in every timestep, which basically involves merging the updated features with the prices catalog.

In [ ]:
fcst.fit(series_with_prices, static_features=['static_0', 'product_id'])

Forecast(models=[LGBMRegressor, XGBRegressor], freq=<Day>, lag_features=['lag-7', 'expanding_mean_lag-1', 'rolling_mean_lag-7_window_size-14'], date_features=[], num_threads=2)

The features used for training are stored in `Forecast.ts.features_order_`, as you can see `price` was used for training.

In [ ]:
fcst.ts.features_order_

['static_0',
 'product_id',
 'price',
 'lag-7',
 'expanding_mean_lag-1',
 'rolling_mean_lag-7_window_size-14']

So in order to update the price in each timestep we just call `Forecast.predict` with our forecast horizon and pass the prices catalog as a dynamic dataframe.

In [ ]:
preds = fcst.predict(7, dynamic_dfs=[prices_catalog])
preds

,ds,LGBMRegressor,XGBRegressor
unique_id,,,
id_00,2001-05-15,42.829567,42.621590
id_00,2001-05-16,49.946924,50.820297
id_00,2001-05-17,2.186703,1.502528
id_00,2001-05-18,10.267976,9.736752
id_00,2001-05-19,18.666022,18.227610
...,...,...,...
id_99,2001-05-17,44.410242,43.474300
id_99,2001-05-18,2.002115,1.789129
id_99,2001-05-19,9.074003,9.035077


### Custom predictions
As you may have noticed `Forecast.predict` can take a `predict_fn` and `predict_fn_kwargs`. By default the predict method repeats the static features and updates the transformations and the date features. If you have dynamic features you can pass them as a list to `Forecast.predict` in the `dynamic_dfs` argument. However, if you want to do something else, you can define your own function which will take:

* The trained model.
* The updated features (static + transformations + date features).
* A list of dataframes with the dynamic features.
* The order of the features the model was trained on.
* Additional keyword arguments passed to `Forecast.predict`.

Here's an example:

Suppose that we want to scale our predictions so that our series are updated with these scaled values. We can achieve that with the following.

In [ ]:
def scaling_predict_fn(
    model,
    new_x,
    dynamic_dfs,
    features_order,
    scale_factor,
) -> np.ndarray:
    new_x = new_x[features_order]
    predictions = model.predict(new_x)
    return scale_factor * predictions

And now we just pass this function to `Forecast.predict`.

In [ ]:
fcst = Forecast(models, freq='D', lags=[7])
fcst.fit(series)

scale_factor = 1.1
preds = fcst.predict(2, predict_fn=scaling_predict_fn, scale_factor=scale_factor)
preds

,ds,LGBMRegressor,XGBRegressor
unique_id,,,
id_00,2001-05-15,46.298462,47.070534
id_00,2001-05-16,54.772835,55.468819
id_01,2001-05-15,12.808338,12.969510
id_01,2001-05-16,16.426544,16.559385
id_02,2001-05-15,16.339826,16.399563
...,...,...,...
id_97,2001-05-16,38.865211,39.316402
id_98,2001-05-15,8.268328,8.267344
id_98,2001-05-16,10.476819,10.496795


In [ ]:
#|hide
fcst.ts._predict_setup()

for attr in ('head', 'tail'):
    new_x = fcst.ts._update_features().drop(columns='ds')
    original_preds = fcst.models_[0].predict(new_x)
    
    expected = scale_factor * original_preds
    actual = getattr(preds.groupby('unique_id')[models[0].__class__.__name__], attr)(1).values
    np.testing.assert_equal(expected, actual)
    
    fcst.ts._update_y(actual)

### Cross validation

If we would like to know how good our forecast will be for a specific model and set of features then we can perform cross validation. What cross validation does is take our data and split it in two parts, where the first part is used for training and the second one for validation. Since the data is time dependant we usually take the last *x* observations from our data as the validation set.

This process is implemented in `Forecast.cross_validation`, which takes our data and performs the process described above for `n_windows` times where each window is of size `window_size`. For example, if we have 100 samples and we want to perform 2 backtests each of size 14, the splits will be as follows:

1. Train: 1 to 72. Validation: 73 to 86.
2. Train: 1 to 86. Validation: 87 to 100.

In [ ]:
show_doc(Forecast.cross_validation)

---

### Forecast.cross_validation

>      Forecast.cross_validation (data:pandas.core.frame.DataFrame,
>                                 n_windows:int, window_size:int,
>                                 id_col:str='index', time_col:str='ds',
>                                 target_col:str='y',
>                                 static_features:Optional[List[str]]=None,
>                                 dropna:bool=True,
>                                 keep_last_n:Optional[int]=None, dynamic_dfs:Op
>                                 tional[List[pandas.core.frame.DataFrame]]=None
>                                 , predict_fn:Optional[Callable]=None,
>                                 **predict_fn_kwargs)

Creates `n_windows` splits of `window_size` from `data`, trains the model
on the training set, predicts the window and merges the actuals and the predictions
in a dataframe.

Returns a dataframe containing the datestamps, actual values, train ends and predictions.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| data | DataFrame |  | time series |
| n_windows | int |  | number of windows to evaluate |
| window_size | int |  | test size in each window |
| id_col | str | index | column that identifies each serie, can also be the index. |
| time_col | str | ds | column with the timestamps |
| target_col | str | y | column with the series values |
| static_features | typing.Optional[typing.List[str]] | None | column names of the features that don't change in time |
| dropna | bool | True | drop rows with missing values created by lags |
| keep_last_n | typing.Optional[int] | None | keep only this many observations of each serie for computing the updates |
| dynamic_dfs | typing.Optional[typing.List[pandas.core.frame.DataFrame]] | None | future values for dynamic features |
| predict_fn | typing.Optional[typing.Callable] | None | custom function to compute predictions |
| predict_fn_kwargs |  |  |  |

In [ ]:
n_windows = 2
window_size = 14

backtest_results = fcst.cross_validation(series, n_windows, window_size)
backtest_results.head()

,ds,cutoff,y,LGBMRegressor,XGBRegressor
unique_id,,,,,
id_00,2001-04-17,2001-04-16,40.499332,41.412064,41.445278
id_00,2001-04-18,2001-04-16,50.888323,49.478838,49.127155
id_00,2001-04-19,2001-04-16,0.121812,2.051727,2.088633
id_00,2001-04-20,2001-04-16,10.987977,10.017578,10.022248
id_00,2001-04-21,2001-04-16,16.370385,18.710617,18.389297


We can aggregate these by date to get a rough estimate of how our model is doing.

In [ ]:
agg_results = backtest_results.groupby('ds').mean()
agg_results.head()

,y,LGBMRegressor,XGBRegressor
ds,,,
2001-04-17,16.123231,16.208721,16.171181
2001-04-18,15.213920,15.131302,15.151584
2001-04-19,16.985699,17.130627,17.121136
2001-04-20,18.068340,18.032494,18.075195
2001-04-21,18.200609,18.094151,18.183397


We can also compute the error for each model.

In [ ]:
{model: np.round(mean_squared_error(backtest_results['y'], backtest_results[model]), 2)
 for model in backtest_results.columns.drop(['ds', 'y', 'cutoff'])}

{'LGBMRegressor': 0.92, 'XGBRegressor': 0.88}

In [ ]:
#|hide
fcst = Forecast(lgb.LGBMRegressor(), freq='D', lags=[7, 14])
backtest_results = fcst.cross_validation(
    non_std_series,
    n_windows,
    window_size,
    id_col='some_id',
    time_col='time',
    target_col='value',
    static_features=['static_0', 'static_1'],
)
renamer = {'some_id': 'unique_id', 'time': 'ds', 'value': 'y'}
backtest_results = backtest_results.rename(columns=renamer).set_index('unique_id')
renamed = non_std_series.rename(columns=renamer).set_index('unique_id')
manual_results = []
for cutoff, train, valid in backtest_splits(renamed, n_windows, window_size, 1):
    fcst.fit(train, static_features=['static_0', 'static_1'])
    pred = fcst.predict(window_size)
    res = valid[['ds', 'y']].copy()
    res['cutoff'] = cutoff
    res = res[['ds', 'cutoff', 'y']].copy()
    manual_results.append(res.merge(pred, on=['unique_id', 'ds'], how='left'))
manual_results = pd.concat(manual_results)
pd.testing.assert_frame_equal(backtest_results, manual_results)